In [4]:
! pip install bertopic



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.9 MB/s 
     |████████████████████████████████| 88 kB 5.0 MB/s 
     |████████████████████████████████| 5.2 MB 48.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 5.4 MB/s 
     |████████████████████████████████| 4.4 MB 49.3 MB/s 
     |████████████████████████████████| 1.2 MB 40.9 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
     |████████████████████████████████| 636 kB 63.6 MB/s 
     |████████████████████████████████| 6.6 MB 39.5 MB/s 
     |████████████████████████████████| 1.1 MB 64.6 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342089 sha256=ab6666d9cccab7e717d8e1281c90f00518fbdda956564f4b0df124996ae1005c
  Stored in directory: /root/.cache

In [5]:

from bertopic import BERTopic
import pandas as pd



In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import pandas as pd
import re
import gensim
from nltk.stem import WordNetLemmatizer

punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
   # tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#)', ' ', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    #tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    #tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
  # tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    tweet = re.sub(' amp ', '&', tweet)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    # tweet = remove_users(tweet)
    # tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub(' & ', '', tweet)
    return tweet

def tokenize_tweets(df):
    df['tokens'] = df.tweet.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

In [64]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/df_nlp_com3.csv",  lineterminator='\n')


### Topic Modelling prime cinque comunità per numero di utenti

In [9]:
df1_com0 = df1[df1["community"]=="community_0"]
df1_com1 = df1[df1["community"]=="community_1"]
df1_com2 = df1[df1["community"]=="community_2"]
df1_com3 = df1[df1["community"]=="community_3"]
df1_com4 = df1[df1["community"]=="community_4"]


In [10]:
joinDf = [df1_com0, df1_com1, df1_com2, df1_com3, df1_com4]

In [11]:
df = pd.concat(joinDf)

In [12]:
df = df.reset_index(drop=True)

In [13]:
df

,id,conversation_id,created_at,date,time,timezone,user_id,name,place,tweet,...,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,community,original_community_extension
0,1386099088056197130,1386076099130580992,2021-04-25 00:25:49+01:00,2021-04-25,00:25:49,100,1656076950,Hank P,NaN,@POTUS This is what leadership looks like. Tha...,...,NaN,NaN,NaN,"[{'screen_name': 'POTUS', 'name': 'President B...",NaN,NaN,NaN,NaN,community_0,80
1,1406301414331064320,1406273372309397504,2021-06-19 18:22:39+01:00,2021-06-19,18:22:39,100,4272749484,DianeE,NaN,@FLOTUS Oh my 😢😢😢 only those who have loved an...,...,NaN,NaN,NaN,"[{'screen_name': 'FLOTUS', 'name': 'Jill Biden...",NaN,NaN,NaN,NaN,community_0,80
2,1378822778250358787,1378806691760726016,2021-04-04 22:32:22+01:00,2021-04-04,22:32:22,100,4272749484,DianeE,NaN,@vicketty @ldsconf @OaksDallinH After 40yrs as...,...,NaN,NaN,NaN,"[{'screen_name': 'vicketty', 'name': 'G-Mama',...",NaN,NaN,NaN,NaN,community_0,80
3,1391093026731139073,1391093026731139072,2021-05-08 19:09:57+01:00,2021-05-08,19:09:57,100,4272749484,DianeE,NaN,Real Conservatives voted for #JoeBiden 🇺🇸🇺🇸🇺🇸 ...,...,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,community_0,80
4,1382430563294801921,1382409575668854784,2021-04-14 21:28:25+01:00,2021-04-14,21:28:25,100,4272749484,DianeE,NaN,@POTUS @POTUS #JoeBiden please listen to those...,...,NaN,NaN,NaN,"[{'screen_name': 'POTUS', 'name': 'President B...",NaN,NaN,NaN,NaN,community_0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,1378071040291651588,1349021895048515584,2021-04-02 20:45:13+01:00,2021-04-02,20:45:13,100,1261015410499039233,@Mohammed2030✍🦅,NaN,@USAbilAraby @abotamam73 @SGCXQ9mi4qWx6KR @AlH...,...,NaN,NaN,NaN,"[{'screen_name': 'USAbilAraby', 'name': 'الخار...",NaN,NaN,NaN,NaN,community_4,47
604,1369769731486588928,1349021895048515584,2021-03-10 21:58:47+00:00,2021-03-10,21:58:47,100,1261015410499039233,@Mohammed2030✍🦅,NaN,@USAbilAraby @abotamam73 @KSAMOFA @SGCXQ9mi4qW...,...,NaN,NaN,NaN,"[{'screen_name': 'USAbilAraby', 'name': 'الخار...",NaN,NaN,NaN,NaN,community_4,47
605,1400875228738244609,1349021895048515584,2021-06-04 19:00:56+01:00,2021-06-04,19:00:56,100,1261015410499039233,@Mohammed2030✍🦅,NaN,@Bdryahalshrhan @USAbilAraby @abotamam73 @SGCX...,...,NaN,NaN,NaN,"[{'screen_name': 'Bdryahalshrhan', 'name': 'بد...",NaN,NaN,NaN,NaN,community_4,47
606,1365048285271056385,1365046353764765696,2021-02-25 21:17:27+00:00,2021-02-25,21:17:27,100,1272002466649964549,Bubby Sadie,NaN,@NBCNews Saving Americans - thank god for Joe ...,...,NaN,NaN,NaN,"[{'screen_name': 'NBCNews', 'name': 'NBC News'...",NaN,NaN,NaN,NaN,community_4,47


In [38]:
df = df.drop_duplicates()

In [39]:
df["tweet"] = df.tweet.astype("str")

In [40]:
import nltk


In [41]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [42]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)


In [43]:
df.value_counts()

Series([], dtype: int64)

In [44]:
df.community.value_counts()

community_0    206
community_4    159
community_2    119
community_3    110
community_1     14
Name: community, dtype: int64

In [45]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()

In [46]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [47]:
topic_model.get_topic_info()


,Topic,Count,Name
0,2,374,2_joebiden_biden_potus_gop
1,4,67,4_opinion_write_biden_putin
2,5,47,5_statedept_secblinken_whitehouse_president
3,7,34,7_israel_break_netanyahu_biden
4,6,26,6_syria_iran_opinion_write
5,-1,18,-1_military_biden_egypt_sisi
6,3,16,3_canadafp_eng_justintrudeau_canadianforces
7,0,15,0_muslimban_lottery_winners_diversity
8,1,11,1_nasa_mar_nasapersevere_space


In [48]:
topic_model.visualize_barchart()


In [49]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)


In [51]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


In [27]:
 topic_model.get_representative_docs(0)


['dear president potus chance lottery winners refuse section pursuant presidential proclamation muslimban diversity visa bidentakeaction joebiden godblessamerica',
 'dear president potus chance lottery winners refuse section pursuant presidential proclamation muslimban diversity visa bidentakeaction joebiden godblessamerica',
 'dear president potus chance lottery winners refuse section pursuant presidential proclamation muslimban diversity visa bidentakeaction joebiden godblessamerica']

### Topic Modelling su tutti i tweet del terzo semestre

In [28]:
df1 = df1.drop_duplicates()

In [29]:
df1["tweet"] = df1.tweet.astype("str")

In [30]:
df1["preprocessed_tweet"] = df1["tweet"].apply(preprocess_tweet)


In [31]:
tweets = df1.preprocessed_tweet.to_list()
timestamp = df1.date.to_list()

In [32]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [33]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,7244,-1_biden_joebiden_trump_potus
1,0,711,0_covid_vaccine_vaccines_mask
2,1,696,1_border_bordercrisis_immigration_crisis
3,2,434,2_israel_gaza_palestinians_palestine
4,3,276,3_china_ccp_crown_connection
5,4,256,4_putin_russia_russian_ukraine
6,5,242,5_joebiden_oscarvreuenthal_joebide_skkyyyyyyyyyyy
7,6,236,6_kamalaharris_joebiden_potus_days
8,7,230,7_truth_racism_capitolriots_facts
9,8,225,8_iran_bomb_democlicans_republicrats


In [34]:
topic_model.visualize_barchart()

In [35]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [36]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

### Topic Modelling sulle prime cinque comunità per numero di tweets

In [65]:
df1['community'].value_counts()

community_1013    408
community_850     391
community_1001    276
community_171     253
community_508     218
                 ... 
community_770       1
community_599       1
community_802       1
community_131       1
community_1041      1
Name: community, Length: 1098, dtype: int64

In [66]:
df1_com1013 = df1[df1["community"]=="community_1013"]
df1_com850 = df1[df1["community"]=="community_850"]
df1_com1001 = df1[df1["community"]=="community_1001"]
df1_com171 = df1[df1["community"]=="community_171"]
df1_com508 = df1[df1["community"]=="community_508"]



In [67]:
joinDf = [df1_com1013, df1_com850, df1_com1001, df1_com171, df1_com508]

In [68]:
df = pd.concat(joinDf)

In [69]:
df = df.reset_index(drop=True)

In [70]:
df = df.drop_duplicates()

In [71]:
df["tweet"] = df.tweet.astype("str")

In [72]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)

In [73]:
df.community.value_counts()

community_1013    408
community_850     391
community_1001    276
community_171     253
community_508     218
Name: community, dtype: int64

In [74]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()



In [75]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

In [76]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,234,-1_biden_administration_say_president
1,2,233,2_truth_racism_trump_capitolriots
2,8,229,8_cnn_msnbc_bbc_cbs
3,16,170,16_joebiden_jensucky_biden_say
4,4,103,4_brave_page_coronavirus_racism
5,20,96,20_border_crisis_migrants_mexico
6,9,72,9_iran_administration_biden_nuclear
7,17,64,17_tax_president_plan_infrastructure
8,7,42,7_hamas_israel_administration_israeli
9,14,40,14_yemen_houthis_iran_ksa


In [77]:
topic_model.visualize_barchart()

In [78]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

In [79]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)